In [ ]:
# importing the neccessary libraries to collab
import pandas as pd
import numpy as np
!pip install XlsxWriter
import xlsxwriter
import openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.1/165.1 kB 3.3 MB/s eta 0:00:00


In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import  statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

In [ ]:
datasets=['Public Underwriters Grp',
 'California Cas Grp',
 'Celina Mut Grp',
 'Federal Ins Co Grp',
 'Employers Mut Co Of Des Moines',

 'West Bend Mut Ins Grp',
 'Grange Ins Asn Grp',
 'Secura Ins Co',
 'Island Ins Cos Grp',
 'Kentucky Farm Bureau Mut Ins Grp',
 'Farmers Automobile Grp',
 'Germania Ins Grp',
 'State Farm Mut Grp',
 'United Services Automobile Asn Grp',
 'Erie Ins Exchange Grp',
 'Farmers Alliance Mut & Affiliates',
 'Oklahoma Farm Grp',
 'Canal Ins Co Grp',
 'Pennsylvania Natl Ins Grp',
 'NC Farm Bureau Ins Grp',
 'Florists Mut Ins Grp',
 'FL Farm Bureau Grp',
 'Grinnell Mut Grp',
 'Cameron Mut Grp',

 'American Contractors Ins Grp',
 'Mutual Benefit Grp',
 'Oregon Mut Grp',
 'Philadelphia Ind Ins Co & Aff',
 'Tenn Farmers Mut',
 'New Jersey Manufacturers Grp',
 'Columbia Ins Grp',
 'Amguard Norguard & Eastguard Grp',
 'Farm Bureau Grp',
 'Millers Mut Ins Assoc',
 'Protective Ins Grp',
 'Lumber Ins Cos',
 'Agway Ins Co',
 'Antilles Ins Co',
 'American Vehicle Ins Co',
 'First Nonprofit Mut Ins Co',
 'Eveready Ins Co',
 'Federated Rural Electric Ins Corp',
 'Yasuda Fire & Marine Ins Co Of Amer',
 'Homestead Ins Co',
 'THE Ins Co',
 'Badger Mut Ins Co',
 'Partners Mut Ins Co',
 'Brethren Mut Ins Co',
 'Brotherhood Mut Ins Co',
 'Chicago Mut Ins Co',
 'Colorado Farm Bureau Mut Ins Co',
 'Farmers Mut Ins Co Of NE',
 'Fitchburg Mut Ins Co',
 'Goodville Mut Cas Co',
 'Hastings Mut Ins Co',
 'IMT Ins Co Mut',
 'Interboro Mut Ind Ins Co',
 'Distributors Ins Co',

 'Michigan Millers Mut Ins Co',
 'Pennsylvania Lumbermens Mut Ins',
 'Preferred Mut Ins Co',
 'Standard Mut Ins Co',

 'American Mining Ins Co Inc',

 'Cooperativa D Seguros Multiples PR',

 'Church Mut Ins Co',
 'Virginia Mut Ins Co',
 'Bell United Ins Co',
 'Springfield Fire & Cas Co',
 'Midstates Rein Corp',
 'Vanliner Ins Co',
 'Transit Mut Ins Corp Of WI',
 'Wellington Ins Co',
 'Midwest Family Mut Ins Co',
 'National American Ins Co',
 'State-Wide Ins Co',
 'Lancer Ins Co',
 'Harco Natl Ins Co',
 'Housing Authority Rrg Inc',
 'Century-Natl Ins Co',
 'Wisconsin Mut Ins Co',


 'Continental Natl Ind Co',
 'Farmers Union Mut Ins Co',
 'Triangle Ins Co Inc',
 'Transguard Ins Co Of Amer Inc',
 'Old American Cty Mut Fire Ins Co',
 'Mountain West Farm Bu Mut Ins Co',
 'Haulers Ins Co Inc',
 'Nichido Fire & Marine Ins Co Ltd',
 'Farmers Union Mut Ins Co',
 'Alliance Mut Ins Co',
 'British Amer Ins Co',
 'Capacity Ins Co',
 'First American Specilaty Ins Co',
 'Center Mut Ins Co',
 'Sirius Amer Ins Co',
 'Daily Underwriters Of Amer',

 'Nonprofits Ins Assn Interins',
 'Governmental Interins Exch',
 'Samsung Fire & Marine Ins Co Ltd',
 'Evergreen USA RRG Inc',
 'Alaska Nat Ins Co',

 'Seguros Triples Inc',
 'American Resources Ins Co Inc',
 'San Antonio Reins Co',

 'College Liability Ins Co Ltd RRG']



In [ ]:
#creating a development traingle
# A general function that can be used for developing any triangle in actuarial science but impt to remember that the data is arrranged in the ascending order in a sorted manner
def d_triangle(df,accident_yr,reported_yr,parameter):

  columns=[]
  max=0
  for i in df[accident_yr].unique():
    a=df.groupby(accident_yr)
    a=a.get_group(i)
    l=len(a)
    if max<l:
      max=l
  final=[]
  for i in range(1,max+1):
    columns.append(12*(i))
  j=0
  for i in df[accident_yr].unique():
    check=1
    lis=[]
    while check and j<len(df):
      if j+1 <= len(df):
        lis.append(df.loc[j,parameter])
      if j+1 >= len(df):
        check=0
      else:
        curr=df[accident_yr][j]
        next=df[accident_yr][j+1]
      if next != curr:
        check=0
      j=j+1
    if (len(lis)<max):
      for i in range(0,max-len(lis)):
        lis.append(np.NaN)
    final.append(lis)
  bf=pd.DataFrame(final,columns=columns)
  bf["Accident Year"]=df[accident_yr].unique()
  first_column = bf.pop('Accident Year')
  bf.insert(0,"Accident Year",first_column)
  return bf





In [ ]:
def cumm_to_incremental(initial,accident_yr,dev_year,parameter):
  li=[]
  for i in range(0,len(initial)):
    if(initial.loc[i,accident_yr]!=initial.loc[i,dev_year]):
      li.append(initial.loc[i,parameter]-initial.loc[i-1,parameter])
    else:
      li.append(initial.loc[i,parameter])
  return li

In [ ]:
def paid_inc_col_ratios(initial,accident_year,dev_year,inc_paid_claims):
  li=[]
  for i in range(0,len(initial)):
    if (initial.loc[i,accident_year]!=initial.loc[i,dev_year]) and initial.loc[i-1,inc_paid_claims]!=0:
       li.append(initial.loc[i,inc_paid_claims]/initial.loc[i-1,inc_paid_claims])
    else:
      li.append(0)
  return li

In [ ]:
def case_col_ratios(initial,accident_year,dev_year,cumm_case_reserves):
  li=[]
  for i in range(0,len(initial)):
    if (initial.loc[i,accident_year]!=initial.loc[i,dev_year]) and initial[cumm_case_reserves][i-1]!=0:
          li.append(initial[cumm_case_reserves][i]/initial[cumm_case_reserves][i-1])
    else:
      li.append(0)
  return li

In [ ]:
def fill_triangles_case_outstanding(case_outstanding, case_ratio):
  df=case_outstanding
  for i in range(0,len(df)):
    for j in range(1,len(case_outstanding.columns)):
      if df.isnull().loc[i,df.columns[j]]:
         df.loc[i,df.columns[j]]=(round(case_ratio.loc[i,case_ratio.columns[j-1]]*df.loc[i,df.columns[j-1]],3))
  return df

In [ ]:
#inc_paid_tringle_development:
def fill_triangles_paid_triangle(case_outstanding,inc_paid_tr,paid_tr):
  columns=list(case_outstanding.columns)
  for i in range(1,len(case_outstanding)):
    for j in range(1,len(columns)):
      if inc_paid_tr.isnull().iloc[i,j]:
        inc_paid_tr.loc[i,columns[j]]=(paid_tr.loc[i,paid_tr.columns[j-1]]*case_outstanding.loc[i].at[columns[j-1]])
  return inc_paid_tr

In [ ]:
a=[]
for i in datasets:
  name=i
  li=[]
  print("The name of the company is:",i)
  initial=pd.read_csv('/content/drive/MyDrive/Colab resources/CsOutstandingProject/gen_for_collab/'+i+'.csv')
  initial.dropna()
  final=pd.read_csv('/content/drive/MyDrive/Colab resources/CsOutstandingProject/Gen_by_code not final/good_data_sets/'+i+'.csv')

  Accident_yr="AccidentYear"
  Reported_yr="DevelopmentYear"
  Reported_claims="Cummulative_loss"
  Paid_Claims="Cummulative_paid_loss"
  Case_outstanding="Cummulative_case_outstanding"

  f_triangle=d_triangle(final,Accident_yr,Reported_yr,Paid_Claims)
  print(f_triangle)
  initial["incremental_paid_claims"]=cumm_to_incremental(initial,Accident_yr,Reported_yr,Paid_Claims)
  initial["inc_paid_ratios"]=paid_inc_col_ratios(initial,Accident_yr,Reported_yr,"incremental_paid_claims")
  initial["Case_Outstanding_ratios"]=case_col_ratios(initial,"AccidentYear","DevelopmentYear","Cummulative_case_outstanding")
  inc_paid_tr=d_triangle(initial,"AccidentYear","DevelopmentYear","incremental_paid_claims")
  print(inc_paid_tr)
  case_outstanding_tr=d_triangle(initial,"AccidentYear","DevelopmentYear","Cummulative_case_outstanding")
  print(case_outstanding_tr)
  initial=initial.where(initial["AccidentYear"]!=initial["DevelopmentYear"]).dropna().reset_index()
  final["incremental_paid_claims"]=cumm_to_incremental(final,Accident_yr,Reported_yr,Paid_Claims)
  final["inc_paid_ratios"]=paid_inc_col_ratios(final,Accident_yr,Reported_yr,"incremental_paid_claims")
  final["Case_Outstanding_ratios"]=case_col_ratios(final,"AccidentYear","DevelopmentYear","Cummulative_case_outstanding")
  final=final.where(final["AccidentYear"]!=final["DevelopmentYear"]).dropna().reset_index()
  final["lag"]=final["DevelopmentYear"]-final["AccidentYear"]
  initial["lag"]=initial["DevelopmentYear"]-initial["AccidentYear"]+1
  train=initial[["AccidentYear","lag"]]
  train["AccidentYear"]=train["AccidentYear"]-train.iloc[0,0]+1
  train_target=initial["Case_Outstanding_ratios"]
  test=final[final['DevelopmentYear']>1997][["AccidentYear","lag"]]
  test["AccidentYear"]=test["AccidentYear"]-train.iloc[0,0]+1
  test_actual=final[final['DevelopmentYear']>1997]["Case_Outstanding_ratios"]
  test_actual.dropna(axis=0,inplace=True)
  test.dropna(axis=0,inplace=True)
  print(train_target)
  print(train)
  model=sm.GLM(train_target,train,family=sm.families.Gamma(link=sm.families.links.InversePower()))
  results=model.fit()
  Results_case=results.predict(test)
  train=initial[["AccidentYear","lag"]]
  train["AccidentYear"]=train["AccidentYear"]-train.iloc[0,0]+1
  train_target=initial["inc_paid_ratios"]
  test=final[final['DevelopmentYear']>1997][["AccidentYear","lag"]]
  test["AccidentYear"]=test["AccidentYear"]-train.iloc[0,0]+1
  test_actual=final[final['DevelopmentYear']>1997]["inc_paid_ratios"]
  test_actual.dropna(axis=0,inplace=True)
  test.dropna(axis=0,inplace=True)
  print(train_target)
  print(train)
  model=sm.GLM(train_target,train,family=sm.families.Gamma(link=sm.families.links.InversePower()))
  results=model.fit()
  Results_paid=results.predict(test)
  test["Case_Outstanding_ratios"]=Results_case
  test["inc_paid_ratios"]=Results_paid
  initial=initial[["AccidentYear","DevelopmentYear","lag","Case_Outstanding_ratios","inc_paid_ratios"]]
  merged_df = pd.merge(initial,test, how="outer")
  merged_df=merged_df.sort_values(by=["AccidentYear","DevelopmentYear"])
  merged_df.reset_index(inplace=True)
  case_ratio_tr=d_triangle(merged_df,"AccidentYear","DevelopmentYear","Case_Outstanding_ratios")
  columns=["AccidentYear"]
  for i in case_ratio_tr.columns.tolist()[1:]:
    columns.append(i+12)
  case_ratio_tr.columns=columns
  print(case_ratio_tr)
  tf=fill_triangles_case_outstanding(case_outstanding_tr,case_ratio_tr)
  paid_ratio_tr=d_triangle(merged_df,"AccidentYear","DevelopmentYear","inc_paid_ratios")
  print(tf)
  columns=["AccidentYear"]
  for i in paid_ratio_tr.columns.tolist()[1:]:
    columns.append(i+12)
  paid_ratio_tr.columns=columns
  print(paid_ratio_tr)
  nf=fill_triangles_paid_triangle(tf,inc_paid_tr,paid_ratio_tr)
  for i in range(2,len(nf.columns)):
    nf[nf.columns[i]]=nf[nf.columns[i]]+nf[nf.columns[i-1]]
  print(nf)
  print(f_triangle[f_triangle.columns[-1]].sum())
  print( nf[nf.columns[-1]].sum())
  li.append(name)
  li.append(f_triangle[f_triangle.columns[-1]].sum())
  li.append(nf[nf.columns[-1]].sum())
  a.append(li)



The name of the company is: Public Underwriters Grp
   Accident Year   12    24    36    48    60    72    84    96   108   120
0           1989    6    20    21    23    24    24    24    24    24    24
1           1990   52    95   105   117   128   128   128   128   128   128
2           1991  127   258   301   332   372   376   377   377   377   376
3           1992  120   225   277   339   361   370   372   379   386   386
4           1993  427  1005  1111  1263  1306  1346  1361  1376  1416  1416
5           1994  584  1511  1716  1817  1909  1933  1958  1958  1958  1958
6           1995  369   741   931   941  1096  1122  1099  1103  1103  1103
7           1996  294   595   657   704   720   722   721   721   721   721
8           1997  312   517   599   632   694   708   714   714   714   714
   Accident Year   12     24     36     48    60   72   84   96  108
0           1989    6   14.0    1.0    2.0   1.0  0.0  0.0  0.0  0.0
1           1990   52   43.0   10.0   12.0  11.0  

<ipython-input-10-4d57b04586e4>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["AccidentYear"]=train["AccidentYear"]-train.iloc[0,0]+1
<ipython-input-10-4d57b04586e4>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_actual.dropna(axis=0,inplace=True)
/usr/local/lib/python3.10/dist-packages/statsmodels/genmod/generalized_linear_model.py:308: DomainWarning: The InversePower link function does not respect the domain of the Gamma family.
  warnings.warn((f"The {type(family.link).__name__} link function "
<ipython-input-10-4d57b04586e4>:46: SettingWithC

   AccidentYear        24        36        48        60        72        84  \
0        1989.0  2.333333  0.071429  2.000000  0.500000  0.000000  0.000000   
1        1990.0  0.826923  0.232558  1.200000  0.916667  0.000000  0.000000   
2        1991.0  1.031496  0.328244  0.720930  1.290323  0.100000  0.250000   
3        1992.0  0.875000  0.495238  1.192308  0.354839  0.409091 -0.026859   
4        1993.0  1.353630  0.183391  1.433962  0.282895 -0.026492 -0.026845   
5        1994.0  1.587329  0.221143  0.492683 -0.026134 -0.026477 -0.026830   
6        1995.0  1.008130  0.510753 -0.025785 -0.026120 -0.026463 -0.026816   
7        1996.0  1.023810 -0.025446 -0.025772 -0.026106 -0.026449 -0.026801   
8        1997.0 -0.025116 -0.025433 -0.025758 -0.026092 -0.026435 -0.026787   

         96       108       120  
0  0.000000  0.000000 -0.028027  
1  0.000000 -0.027627 -0.028011  
2 -0.027238 -0.027611 -0.027995  
3 -0.027223 -0.027596 -0.027980  
4 -0.027208 -0.027581 -0.027964  
5 -0.

<ipython-input-10-4d57b04586e4>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["AccidentYear"]=train["AccidentYear"]-train.iloc[0,0]+1
<ipython-input-10-4d57b04586e4>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_actual.dropna(axis=0,inplace=True)
/usr/local/lib/python3.10/dist-packages/statsmodels/genmod/generalized_linear_model.py:308: DomainWarning: The InversePower link function does not respect the domain of the Gamma family.
  warnings.warn((f"The {type(family.link).__name__} link function "
<ipython-input-10-4d57b04586e4>:46: SettingWithC

   Accident Year   12          24           36           48           60  \
0           1988  361  905.000000  1171.000000  1440.000000  1677.000000   
1           1989  247  807.000000  1009.000000  1222.000000  1393.000000   
2           1990  154  596.000000   710.000000   762.000000   869.000000   
3           1991  187  361.000000   676.000000   749.000000   785.000000   
4           1992  128  311.000000   399.000000   531.000000   549.000000   
5           1993   45  107.000000   172.000000   176.000000   178.000000   
6           1994   29   71.000000    96.000000   116.000000    98.618825   
7           1995   20   91.000000   103.000000    94.960439    94.788052   
8           1996    8  100.000000    97.893018    97.847745    97.846770   
9           1997    0   -0.065965    -0.067382    -0.067408    -0.067408   

            72           84           96          108          120  
0  1779.000000  1776.000000  1778.000000  1780.000000  1780.000000  
1  1430.000000  1431.0000

...

<ipython-input-10-4d57b04586e4>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["AccidentYear"]=train["AccidentYear"]-train.iloc[0,0]+1
<ipython-input-10-4d57b04586e4>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_actual.dropna(axis=0,inplace=True)
/usr/local/lib/python3.10/dist-packages/statsmodels/genmod/generalized_linear_model.py:308: DomainWarning: The InversePower link function does not respect the domain of the Gamma family.
  warnings.warn((f"The {type(family.link).__name__} link function "
<ipython-input-10-4d57b04586e4>:46: SettingWithC

   Accident Year   12        24        36        48        60        72  \
0           1988  176   529.000  1078.000  1700.000  2390.000  3153.000   
1           1989  219   689.000  1446.000  2330.000  3385.000  4611.000   
2           1990  288   813.000  1701.000  2797.000  4101.000  5613.000   
3           1991  199   610.000  1332.000  2325.000  3728.000  5575.000   
4           1992  235   901.000  2014.000  3524.000  5511.000  8028.000   
5           1993  273   819.000  1799.000  3224.000  5118.000   160.150   
6           1994  403  1232.000  2310.000  3704.000   116.267     3.636   
7           1995  369  1148.000  2543.000    80.074     2.512     0.079   
8           1996  608  2025.000    63.964     2.013     0.063     0.002   
9           1997  861    27.282     0.861     0.027     0.001     0.000   

         84        96       108       120  
0  3989.000  4898.000  5879.000  6932.000  
1  6008.000  7576.000  9315.000   287.889  
2  7334.000  9264.000   287.198     8.872 

<ipython-input-10-4d57b04586e4>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["AccidentYear"]=train["AccidentYear"]-train.iloc[0,0]+1
<ipython-input-10-4d57b04586e4>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_actual.dropna(axis=0,inplace=True)
/usr/local/lib/python3.10/dist-packages/statsmodels/genmod/generalized_linear_model.py:308: DomainWarning: The InversePower link function does not respect the domain of the Gamma family.
  warnings.warn((f"The {type(family.link).__name__} link function "
<ipython-input-10-4d57b04586e4>:46: SettingWithC

   Accident Year   12   24   36   48   60   72   84   96  108  120
0           1988   15   51   36   36   36   36   36   36   36   36
1           1989   12   39   84   43   40   40   40   40   40   40
2           1990   27   36   40   40   40   41   41   41   41   41
3           1991    5   63   83   83   82   82   83   83   83   83
4           1992   46   78  361  355  331  331  331  331  330  330
5           1993   27   31   65   65   65   66   66   66   66   66
6           1994   30  159  135  134  144  144  144  144  144  144
7           1995  149  183  191  203  233  329  329  329  329  329
8           1996   33   51   63   63   63   63   63   63   63   45
9           1997   15   16   18   38   38   40   40   40   29   29
   Accident Year   12     24     36    48    60   72   84   96  108  120
0           1988   15   36.0  -15.0   0.0   0.0  0.0  0.0  0.0  0.0  0.0
1           1989   12   27.0   45.0 -41.0  -3.0  0.0  0.0  0.0  0.0  NaN
2           1990   27    9.0    4.0   0.0   

<ipython-input-10-4d57b04586e4>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["AccidentYear"]=train["AccidentYear"]-train.iloc[0,0]+1
<ipython-input-10-4d57b04586e4>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_actual.dropna(axis=0,inplace=True)
/usr/local/lib/python3.10/dist-packages/statsmodels/genmod/generalized_linear_model.py:308: DomainWarning: The InversePower link function does not respect the domain of the Gamma family.
  warnings.warn((f"The {type(family.link).__name__} link function "
<ipython-input-10-4d57b04586e4>:46: SettingWithC

In [ ]:
len(datasets)

105

In [ ]:
a

[['Public Underwriters Grp', 6826, 4426.299860492792],
 ['California Cas Grp', 6031, 2934.6367076321417],
 ['Celina Mut Grp', 40000, 34638.053141461955],
 ['Federal Ins Co Grp', 745997, 616528.0495587015],
 ['Employers Mut Co Of Des Moines', 388485, 330333.221157277],
 ['West Bend Mut Ins Grp', 107590, 88923.3612928771],
 ['Grange Ins Asn Grp', 24613, 41641.18685871202],
 ['Secura Ins Co', 23714, -11276.152082801036],
 ['Island Ins Cos Grp', 63022, 48049.2863499452],
 ['Kentucky Farm Bureau Mut Ins Grp', 21354, 19199.629535194803],
 ['Farmers Automobile Grp', 90687, 76830.23821568392],
 ['Germania Ins Grp', 1779, 1483.8237812857333],
 ['State Farm Mut Grp', 2226624, 1940845.7593501545],
 ['United Services Automobile Asn Grp', 4601, 3066.523703197606],
 ['Erie Ins Exchange Grp', 525310, 416595.4979107168],
 ['Farmers Alliance Mut & Affiliates', 41489, 33997.199558642315],
 ['Oklahoma Farm Grp', 5528, 4934.394425868695],
 ['Canal Ins Co Grp', 472426, 364209.95004094095],
 ['Pennsylvania 

In [ ]:
df=pd.DataFrame(a,columns=["Name of the company","Actual_ult_cost","Mach_Pred_Cost"])

In [ ]:
df["Error"]=df['Actual_ult_cost']-df['Mach_Pred_Cost']
df["% error"]=df['Error']/df["Actual_ult_cost"]*100

In [ ]:
writer=pd.ExcelWriter('/content/drive/MyDrive/Colab resources/CsOutstandingProject/Mach_learn_reports_for_good_data_sets'+".xlsx",engine='xlsxwriter')
df.to_excel(writer,sheet_name="Mach_Results for good_data_sets")
writer.save()

<ipython-input-15-dbc995c5fe47>:3: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [ ]:
df

,Name of the company,Actual_ult_cost,Mach_Pred_Cost,Error,% error
0,Public Underwriters Grp,6826.0,4426.299860,2399.700140,35.155291
1,California Cas Grp,6031.0,2934.636708,3096.363292,51.340794
2,Celina Mut Grp,40000.0,34638.053141,5361.946859,13.404867
3,Federal Ins Co Grp,745997.0,616528.049559,129468.950441,17.355157
4,Employers Mut Co Of Des Moines,388485.0,330333.221157,58151.778843,14.968861
...,...,...,...,...,...
100,Alaska Nat Ins Co,27091.0,25886.146292,1204.853708,4.447432
101,Seguros Triples Inc,13211.0,9035.576524,4175.423476,31.605658
102,American Resources Ins Co Inc,5124.0,3498.952169,1625.047831,31.714439
103,San Antonio Reins Co,312.0,310.249674,1.750326,0.561002


In [ ]:
sk=StandardScaler()
v1=sk.fit_transform(df[["Actual_ult_cost","Mach_Pred_Cost"]])


In [ ]:
ch=pd.DataFrame(v1,columns=["Actual_ult_cost","Mach_Pred_Cost"])

In [ ]:
mae=mean_absolute_error(ch['Actual_ult_cost'],ch['Mach_Pred_Cost'])

In [ ]:
mse=mean_squared_error(ch['Actual_ult_cost'],ch['Mach_Pred_Cost'])

In [ ]:
mae

0.04795088330122375

In [ ]:
mse

0.03214898506509198

In [ ]:
score=r2_score(df["Actual_ult_cost"],df["Mach_Pred_Cost"])

In [ ]:
score

0.9571808438134309

In [ ]:
df[df["% error"]<10 & df["% error"] > -10 ]["% error"].count()

TypeError: Cannot perform 'rand_' with a dtyped [float64] array and scalar of type [bool]